Code for final model selection using cross validation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

slr = LinearRegression()
mlr = LinearRegression()

# rmses will hold the cross validation root mean squared errors of each model.
rmses = np.zeros((2, 5))

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit both models
    slr.fit(X_train_train[["Length"]], y_train_train)
    mlr.fit(X_train_train[continuous_features], y_train_train)

    ## Use both models to generate predictions on the holdout set
    slr_preds = slr.predict(X_holdout[["Length"]])
    mlr_preds = mlr.predict(X_holdout[continuous_features])

    ## Record the rmses
    rmses[0, i] = root_mean_squared_error(y_holdout, slr_preds)
    rmses[1, i] = root_mean_squared_error(y_holdout, mlr_preds)